# Problem Set 3: Geometry
---
## Setup

In [15]:
# IPython magic
%load_ext autoreload
%autoreload 2
# Matplotlib magic
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
# Imports
import cv2
import matplotlib.pyplot as plt
import numpy as np
# Matplotlib params
plt.rcParams['figure.figsize'] = (14.0, 6.0)
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['figure.titlesize'] = 22
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

---
## 1. Calibration

In [64]:
# Actual points in image A
with open('pts2d-pic_a.txt', 'r') as f:
    pts2d_a = [[int(n) for n in line.split()] for line in f]
# Actual points in 3D space
with open('pts3d.txt', 'r') as f:
    pts3d = [[float(n) for n in line.split()] for line in f]
# Normalized points in image A
with open('pts2d-norm-pic_a.txt', 'r') as f:
    pts2d_norm_a = [[float(n) for n in line.split()] for line in f]
# Normalized points in 3D space
with open('pts3d-norm.txt', 'r') as f:
    pts3d_norm = [[float(n) for n in line.split()] for line in f]

### 1.1 M from least squares

In [65]:
num_points = len(pts3d_norm)
num_variables = 11
# AM = b
A = np.zeros((2*num_points, num_variables), dtype=np.float)
b = np.zeros((2*num_points), dtype=np.float)
for ix, ([u, v], [x, y, z]) in enumerate(zip(pts2d_norm_a, pts3d_norm)):
    A[2*ix, :]   = [x, y, z, 1, 0, 0, 0, 0, -u*x, -u*y, -u*z]
    A[2*ix+1, :] = [0, 0, 0, 0, x, y, z, 1, -v*x, -v*y, -v*z]
    b[2*ix]   = u
    b[2*ix+1] = v

In [66]:
# M = np.linalg.lstsq(A, b)[0]
# M = (A'A)^-1.A'b
M = np.linalg.inv(A.T@A) @ A.T @ b
M = np.append(M, 1).reshape((3, 4))

In [105]:
pt_3d = pts3d_norm[-1]
print('Coordinates of point in 3D:      ({:.4f}, {:7.4f}, {:7.4f})'.format(*pt_3d))
pt_3d.append(1)
us, vs, s = M @ last_point
print('Calculated coordinates in Image: ({:.4f}, {:7.4f})'.format(us/s, vs/s))
u, v = pts2d_norm_a[-1]
print('Actual coordinates in Image:     ({:.4f}, {:7.4f})'.format(u, v))
resid = np.sqrt((u-us/s)**2 + (v-vs/s)**2)
print('Calculated residue:               {:.4f}'.format(resid))

Coordinates of point in 3D:      (1.2323,  1.4421,  0.4506)
Calculated coordinates in Image: (0.1419, -0.4518)
Actual coordinates in Image:     (0.1406, -0.4527)
Calculated residue:               0.0016


### 1.2 M from least squares